# adapters

> Adapter utilities for making plugin managers compatible with the worker system.

In [ ]:
#| default_exp core.adapters

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Iterator, Optional
from cjm_fasthtml_workers.core.protocol import PluginManagerAdapter

## Simple Adapter Factory

In [ ]:
#| export
def create_simple_adapter(
    plugin_manager,  # The plugin manager instance to adapt
    result_adapter: Optional[callable] = None  # Optional function to convert plugin results to dict
) -> PluginManagerAdapter:  # Adapter that satisfies PluginManagerAdapter protocol
    """
    Create a simple adapter for a plugin manager.

    This is a convenience function that wraps a plugin manager object
    to satisfy the PluginManagerAdapter protocol.

    Example:
        >>> from some_plugin_system import PluginManager
        >>> pm = PluginManager()
        >>> adapter = create_simple_adapter(pm)
        >>> # adapter now satisfies PluginManagerAdapter protocol
    """

    class SimpleAdapter:
        """Simple wrapper adapter for plugin managers."""

        def __init__(self, manager, result_fn=None):
            self._manager = manager
            self._result_adapter = result_fn

        def discover_plugins(self) -> list:
            """Discover available plugins."""
            return self._manager.discover_plugins()

        def load_plugin(self, plugin_data: Any, config: Dict[str, Any]) -> None:
            """Load a plugin with configuration."""
            self._manager.load_plugin(plugin_data, config)

        def execute_plugin(self, plugin_name: str, **params) -> Any:
            """Execute a plugin with given parameters."""
            result = self._manager.execute_plugin(plugin_name, **params)

            # Apply result adapter if provided
            if self._result_adapter:
                return self._result_adapter(result)

            return result

        def execute_plugin_stream(self, plugin_name: str, **params) -> Iterator[str]:
            """Execute a plugin with streaming output."""
            return self._manager.execute_plugin_stream(plugin_name, **params)

        def reload_plugin(self, plugin_name: str, config: Optional[Dict[str, Any]] = None) -> None:
            """Reload a plugin with new configuration."""
            self._manager.reload_plugin(plugin_name, config)

        def unload_plugin(self, plugin_name: str) -> None:
            """Unload a plugin to free resources."""
            self._manager.unload_plugin(plugin_name)

        def check_streaming_support(self, plugin_name: str) -> bool:
            """Check if a plugin supports streaming execution."""
            return self._manager.check_streaming_support(plugin_name)

    return SimpleAdapter(plugin_manager, result_adapter)

## Default Result Adapter

In [ ]:
#| export
def default_result_adapter(
    result: Any  # Plugin execution result
) -> Dict[str, Any]:  # Dictionary with text and metadata
    """
    Default adapter for converting plugin results to dictionaries.

    Assumes result has 'text' and 'metadata' attributes.
    """
    if isinstance(result, dict):
        return result

    return {
        'text': getattr(result, 'text', str(result)),
        'metadata': getattr(result, 'metadata', {})
    }

In [ ]:
# Test with dict input
result_dict = {'text': 'Hello', 'metadata': {'key': 'value'}}
default_result_adapter(result_dict)

{'text': 'Hello', 'metadata': {'key': 'value'}}

In [ ]:
# Test with object that has text and metadata attributes
class DummyResult:
    def __init__(self):
        self.text = "Result text"
        self.metadata = {"source": "test"}

default_result_adapter(DummyResult())

{'text': 'Result text', 'metadata': {'source': 'test'}}

In [ ]:
# Test with plain string
default_result_adapter("Plain text result")

{'text': 'Plain text result', 'metadata': {}}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()